In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# --- Load data ---
PATH_QUERY_TRAIN_DATA = '/content/drive/MyDrive/AIR/subtask4b_query_tweets_train.tsv'
PATH_QUERY_DEV_DATA   = '/content/drive/MyDrive/AIR/subtask4b_query_tweets_dev.tsv'
PATH_COLLECTION_DATA  = '/content/drive/MyDrive/AIR/subtask4b_collection_data.pkl'

df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep='\t')
df_query_dev   = pd.read_csv(PATH_QUERY_DEV_DATA, sep='\t')
df_collection  = pd.read_pickle(PATH_COLLECTION_DATA)  # this is a dict



In [3]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [4]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [5]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 2. Representation learning Approach

### 2.1 SBERT

In [6]:
import numpy as np
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [7]:
# Combine title + abstract for each document
df_collection['full_text'] = (
    df_collection['title'].fillna('') + ' ' + df_collection['abstract'].fillna('')
)

# SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Optionally switch to 'allenai/specter'

# Encode documents
print("🔍 Encoding documents...")
paper_ids = df_collection['cord_uid'].tolist()
paper_texts = df_collection['full_text'].tolist()
paper_embeddings = model.encode(paper_texts, convert_to_tensor=True, show_progress_bar=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔍 Encoding documents...


Batches:   0%|          | 0/242 [00:00<?, ?it/s]

In [8]:
results = []

for _, row in tqdm(df_query_train.iterrows(), total=len(df_query_train)):
    tweet_id = str(row['post_id'])
    tweet_text = row['tweet_text']

    query_embedding = model.encode(tweet_text, convert_to_tensor=True)
    cosine_scores = util.cos_sim(query_embedding, paper_embeddings)[0]
    top_indices = cosine_scores.topk(5).indices.cpu().numpy()
    top_doc_ids = [paper_ids[i] for i in top_indices]

    results.append({
        "id": tweet_id,
        "doc_ids": top_doc_ids
    })

# Save result file
with open("/content/drive/MyDrive/AIR/2_nlp_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Saved top-5 results to 2_nlp_results.json")

100%|██████████| 12853/12853 [09:58<00:00, 21.48it/s]


✅ Saved top-5 results to 2_nlp_results.json


In [9]:
def compute_mrr_at_5(predicted, ground_truth):
    mrr_total = 0.0
    query_count = 0

    for post_id, correct_doc_id in ground_truth.items():
        if post_id not in predicted:
            continue

        correct_doc_id = correct_doc_id.strip().lower()
        ranked_docs = [doc.strip().lower() for doc in predicted[post_id][:5]]

        for rank, doc_id in enumerate(ranked_docs, start=1):
            if doc_id == correct_doc_id:
                mrr_total += 1 / rank
                break

        query_count += 1

    return mrr_total / query_count if query_count > 0 else 0.0

In [10]:
# Load submission
with open("/content/drive/MyDrive/AIR/2_nlp_results.json", "r") as f:
    submission = json.load(f)

predicted = {str(entry["id"]): entry["doc_ids"] for entry in submission}

# Prepare ground truth from df_query_train
ground_truth = dict(zip(df_query_train['post_id'].astype(str), df_query_train['cord_uid'].astype(str)))

# Evaluate
mrr = compute_mrr_at_5(predicted, ground_truth)
print(f"✅ MRR@5 (SBERT): {mrr:.4f}")

✅ MRR@5 (SBERT): 0.4620


### 2.2 SBERT + Loss

In [33]:
pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 812.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-p

In [30]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep='\t')
df_query_dev   = pd.read_csv(PATH_QUERY_DEV_DATA, sep='\t')
df_collection  = pd.read_pickle(PATH_COLLECTION_DATA)

# If cord_uid isn't index already
if df_collection.index.name != 'cord_uid':
    df_collection = df_collection.set_index('cord_uid')

# Create full_text field
df_collection['full_text'] = (
    df_collection['title'].fillna('') + ' ' + df_collection['abstract'].fillna('')
)

# Filter for matching cord_uid
df_query_filtered = df_query_train[df_query_train['cord_uid'].isin(df_collection.index)]


In [36]:
from sentence_transformers import InputExample
from datasets import Dataset

train_samples = [
    InputExample(
        texts=[row['tweet_text'], df_collection.loc[row['cord_uid']]['full_text']],
        label=1.0
    )
    for _, row in df_query_filtered.iterrows()
]


In [37]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader

# Base model (you can use 'allenai/specter' as well)
model = SentenceTransformer('all-MiniLM-L6-v2')

# DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)

# Loss
train_loss = losses.MultipleNegativesRankingLoss(model)

# Fine-tune
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=100,
    show_progress_bar=True,
    output_path="output/4_2_sbert_finetuned"
)


NameError: name 'Dataset' is not defined

### 2.3 AllenAI

In [20]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep='\t')
df_query_dev   = pd.read_csv(PATH_QUERY_DEV_DATA, sep='\t')
df_collection  = pd.read_pickle(PATH_COLLECTION_DATA)

In [27]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from datasets import Dataset

In [26]:
pip install datasets

In [21]:
df_collection['full_text'] = (
    df_collection['title'].fillna('') + ' ' +
    df_collection['abstract'].fillna('') + ' ' +
    df_collection['journal'].fillna('') + ' ' +
    df_collection['authors'].fillna('')
)

In [28]:
if df_collection.index.name != 'cord_uid':
    df_collection = df_collection.set_index('cord_uid')

train_samples = [
    InputExample(texts=[tweet, doc], label=1.0)
    for tweet, doc in zip(df_query_train['tweet_text'], df_collection.loc[df_query_train['cord_uid']]['full_text'])
]

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1)

NameError: name 'Dataset' is not defined